In [50]:
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client


server_params = StdioServerParameters(
    command="python",
    args=["../src/mcp_arxiv_server.py"],
    env=None
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        try:
            prompts = await session.list_prompts()
            print(f"Available prompts: {[p.name for p in prompts.prompts]}")

            for prompt in prompts.prompts:
                prompt_name = prompt.name
                prompt_description = prompt.description
                prompt_args = prompt.arguments
                prompt_template = await session.get_prompt(
                    name = prompt.name,
                    arguments = {'topic': '<PLACEHOLDER>'}
                )
                print(f"Prompt name: {prompt.name}, Description: {prompt.description}, Args: {prompt.arguments}")

            resources = await session.list_resources()
            print(f"Available resources: {[r.uri for r in resources.resources]}")

            tools = await session.list_tools()
            print(f"Available tools: {[t.name for t in tools.tools]}")
        except Exception as e:
            print(f"Error calling list_tools: {e}")

Available prompts: ['generate_search_prompt']
Prompt name: generate_search_prompt, Description: 
    Generate a prompt for Claude to find and discuss academic papers on a specific topic.
    , Args: [PromptArgument(name='topic', description=None, required=True), PromptArgument(name='num_papers', description=None, required=False)]
Available resources: [AnyUrl('papers://folders')]
Available tools: ['search_papers', 'extract_info']


In [63]:
prompt_template.messages[0].content.text

"Search for 5 academic papers about '<PLACEHOLDER>' using the search_papers tool. Follow these instructions:\n    1. First, search for papers using search_papers(topic='<PLACEHOLDER>', max_results=5)\n    2. For each paper found, extract and organize the following information:\n       - Paper title\n       - Authors\n       - Publication date\n       - Brief summary of the key findings\n       - Main contributions or innovations\n       - Methodologies used\n       - Relevance to the topic '<PLACEHOLDER>'\n    \n    3. Provide a comprehensive summary that includes:\n       - Overview of the current state of research in '<PLACEHOLDER>'\n       - Common themes and trends across the papers\n       - Key research gaps or areas for future investigation\n       - Most impactful or influential papers in this area\n    \n    4. Organize your findings in a clear, structured format with headings and bullet points for easy readability.\n    \n    Please present both detailed information about eac